In [1]:
# https://competitions.codalab.org/competitions/16652
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from bs4 import BeautifulSoup

In [2]:
base_dir_tr='training/'
base_dir_vl='validation/'

In [3]:
dtr=pd.read_csv(base_dir_tr+'data_train.csv',header=None)
col_names=['country','sku_id','title','category_lvl_1','category_lvl_2','category_lvl_3','short_description','price','product_type']
dtr.columns=col_names

#### conciseness_train
xtr=open(base_dir_tr+'conciseness_train.labels')
concise=[]
for l in xtr:
    concise.append(int(l))
    
#### clarity_train
xtr=open(base_dir_tr+'clarity_train.labels')
clarity=[]
for l in xtr:
    clarity.append(int(l))

dtr['conci']=concise
dtr['clari']=clarity

In [4]:
dvl=pd.read_csv(base_dir_vl+'data_valid.csv',header=None)
col_names=['country','sku_id','title','category_lvl_1','category_lvl_2','category_lvl_3','short_description','price','product_type']
dvl.columns=col_names

In [5]:
#### data manipulation 1

In [6]:
def token_text(x):
    try:
        s1=BeautifulSoup(x, "html5lib").get_text()
    
    except:
        print(x)
        
    s1=BeautifulSoup(x, "html5lib").get_text()
    s1=s1.replace(u'\xa0', u' ')
    tokenizer = RegexpTokenizer(r'\w+')
    word_list=tokenizer.tokenize(s1)
    filtered_words = [word.lower() for word in word_list if word.lower() not in stopwords.words('english')]
    return filtered_words

def to_rnk(Xt,dicr):
    wr=[]
    for x in Xt:
        try:
            wr.append(dicr[x])
        except:
            wr.append(0)
    return wr

In [7]:
dtr.head()

,country,sku_id,title,category_lvl_1,category_lvl_2,category_lvl_3,short_description,price,product_type,conci,clari
0,my,AD674FAASTLXANMY,Adana Gallery Suri Square Hijab – Light Pink,Fashion,Women,Muslim Wear,<ul><li>Material : Non sheer shimmer chiffon</...,49.00,local,1,1
1,my,AE068HBAA3RPRDANMY,Cuba Heartbreaker Eau De Parfum Spray 100ml/3.3oz,Health & Beauty,Bath & Body,Hand & Foot Care,Formulated with oil-free hydrating botanicals/...,128.00,international,1,1
2,my,AN680ELAA9VN57ANMY,Andoer 150cm Cellphone Smartphone Mini Dual-He...,"TV, Audio / Video, Gaming & Wearables",Audio,Live Sound & Stage,<ul> <li>150cm mini microphone compatible for ...,25.07,international,0,1
3,my,AN957HBAAAHDF4ANMY,ANMYNA Complaint Silky Set 柔顺洗发配套 (Shampoo 520...,Health & Beauty,Hair Care,Shampoos & Conditioners,<ul> <li>ANMYNA Complaint Silky Set (Shampoo 5...,118.00,local,1,1
4,my,AR511HBAXNWAANMY,Argital Argiltubo Green Clay For Face and Body...,Health & Beauty,Men's Care,Body and Skin Care,<ul> <li>100% Authentic</li> <li>Rrefresh and ...,114.80,international,1,1


In [8]:
dtr.title=dtr.title.fillna('-')
print('tokenize')
dtr['token_title_des']=dtr.title.apply(token_text)

tokenize


In [9]:
dvl.title=dvl.title.fillna('-')
dvl['token_title_des']=dvl.title.apply(token_text)

word_lis=[]
for ws in dtr.token_title_des:
    word_lis+=ws

In [15]:
wl=pd.Series(word_lis)
wo=pd.DataFrame(wl.value_counts(),columns=['wo'])
wo['rank']=[i+1 for i in range(len(wo))]
wor=wo['rank']

min_words=25000
# min_words=25000
word_rnk_tl=wor[wor<min_words].to_dict()

dtr['rank_title']=dtr.token_title_des.apply(lambda x: to_rnk(x,word_rnk_tl))
dvl['rank_title']=dvl.token_title_des.apply(lambda x: to_rnk(x,word_rnk_tl))

dtr[['sku_id','token_title_des','rank_title','conci','clari']].to_csv(base_dir_tr+'data_title_25k_tr.csv',index=0)
dvl[['sku_id','token_title_des','rank_title']].to_csv(base_dir_vl+'data_title_25k_vl.csv',index=0)

In [12]:
dtr['rank_short_des']=dtr.token_short_des.apply(lambda x: to_rnk(x,word_rnk_sd))
dtr[['sku_id','rank_short_des','conci','clari']].to_csv(base_dir_tr+'data_manipulation_2_train.csv',index=0)

dvl['rank_short_des']=dvl.token_short_des.apply(lambda x: to_rnk(x,word_rnk_sd))
dvl[['sku_id','rank_short_des']].to_csv(base_dir_vl+'data_manipulation_2_valid.csv',index=0)

In [ ]:
#### data 2